In [57]:
import pandas as pd 
import numpy as np 
#import geopandas as gpd 
import sys
import matplotlib.pyplot as plt 
import pickle
import math


In [9]:
#Read CSV file
df = pd.read_csv("census.csv")

#Extract census data from 2016 (year ID = 9)
df = df.drop(df[df.YEAR != 9].index)

#Drop rows for population 14 and younger (agegrp 0-3)
df = df.drop(df[df.AGEGRP < 4].index)

df

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,AGEGRP,TOT_POP,TOT_MALE,TOT_FEMALE,...,HWAC_MALE,HWAC_FEMALE,HBAC_MALE,HBAC_FEMALE,HIAC_MALE,HIAC_FEMALE,HAAC_MALE,HAAC_FEMALE,HNAC_MALE,HNAC_FEMALE
156,50,13,1,Georgia,Appling County,9,4,1205,652,553,...,72,67,3,9,4,3,1,1,4,0
157,50,13,1,Georgia,Appling County,9,5,1073,606,467,...,68,38,7,6,1,3,0,0,4,0
158,50,13,1,Georgia,Appling County,9,6,1149,601,548,...,65,77,7,5,5,1,2,0,4,0
159,50,13,1,Georgia,Appling County,9,7,1053,527,526,...,83,36,8,4,4,5,0,1,5,0
160,50,13,1,Georgia,Appling County,9,8,1097,546,551,...,77,46,7,4,4,1,0,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36190,50,13,321,Georgia,Worth County,9,14,1273,616,657,...,3,4,0,2,0,0,0,0,0,0
36191,50,13,321,Georgia,Worth County,9,15,1025,464,561,...,4,8,0,1,0,0,0,0,0,0
36192,50,13,321,Georgia,Worth County,9,16,676,307,369,...,3,2,1,0,0,0,0,0,0,0
36193,50,13,321,Georgia,Worth County,9,17,405,158,247,...,2,1,0,0,0,0,0,0,0,0


In [93]:
from itertools import product


#Make empty data frame

def expand_grid(dictionary):
       return pd.DataFrame([row for row in product(*dictionary.values())], 
                       columns=dictionary.keys())

dictionary = {'county_fips_code': ['1','3','5','7','9','11','13','15','17','19','21','23','25','27','29','31','33','35','37','39','43','45','47','49','51','53','55','57','59','61'
                                    ,'63','65','67','69','71','73','75','77','79','81','83','85','87','89','91','93','95','97','99','101','103','105','107','109','111','113','115'
                                   ,'117','119','121','123','125','127','129','131','133','135','137','139','141','143','145','147','149','151','153','155','157','159','161','163','165','167','169','171','173','175','177','179','181','183','185','187','189','191','193','195','197','199','201','205','207','209','211','213','215','217','219','221','223','225','227','229','231','233','235','237','239','241','243','245','247','249','251','253','255','257','259','261','263','265','267','269','271','273'
                                    ,'275','277','279','281','283','285','287','289','291','293','295','297','299','301','303','305','307','309','311','313','315','317','319','321'],
              'year': ['2016'],
              'aggrp': ['18-24','25-29','30-34','35-39','40-44','45-49','50-54','55-59','60-64','65+'], 
              'race': ['white','black','asia-pi','hisp-lt','native-am','other'], #other is two or more races
             'gender':['male', 'female'],
             'population':['0']}

pop = expand_grid(dictionary)
pop

,county_fips_code,year,aggrp,race,gender,population
0,1,2016,18-24,white,male,0
1,1,2016,18-24,white,female,0
2,1,2016,18-24,black,male,0
3,1,2016,18-24,black,female,0
4,1,2016,18-24,asia-pi,male,0
...,...,...,...,...,...,...
19075,321,2016,65+,hisp-lt,female,0
19076,321,2016,65+,native-am,male,0
19077,321,2016,65+,native-am,female,0
19078,321,2016,65+,other,male,0


In [94]:
for index, row in df.iterrows():
    county = row['COUNTY']
    #ages 15-19
    if row['AGEGRP'] == 4:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHWA_MALE'])
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHWA_FEMALE'])        
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHBA_MALE'])        
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHBA_FEMALE'])        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHAA_MALE'])        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHAA_FEMALE'])        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['H_MALE'])        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['H_FEMALE'])        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + (math.floor((2.0/5.0)*row['NHIA_MALE']) + math.floor((2.0/5.0)*row['NHNA_MALE']))    
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + (math.floor((2.0/5.0)*row['NHIA_FEMALE']) + math.floor((2.0/5.0)*row['NHNA_FEMALE']))       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHTOM_MALE'])        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + math.floor((2.0/5.0)*row['NHTOM_FEMALE'])        
    
    #ages 20-24
    if row['AGEGRP'] == 5:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '18-24') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 25-29
    if row['AGEGRP'] == 6:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '25-29') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 30-34
    if row['AGEGRP'] == 7:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '30-34') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 35-39
    if row['AGEGRP'] == 8:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '35-39') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 40-44
    if row['AGEGRP'] == 9:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '40-44') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 45-49
    if row['AGEGRP'] == 10:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '45-49') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 50-54
    if row['AGEGRP'] == 11:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '50-54') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 55-59
    if row['AGEGRP'] == 12:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '55-59') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 60-64
    if row['AGEGRP'] == 13:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '60-64') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
    #ages 65+
    if row['AGEGRP'] >= 14:
        #white male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'white') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_MALE']
        #white female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'white') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHWA_FEMALE']      
        #black male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'black') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_MALE']       
        #black female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'black') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHBA_FEMALE']        
        #asia-pi male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_MALE']        
        #asia-pi female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'asia-pi') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHAA_FEMALE']        
        #hisp-lt male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_MALE']        
        #hisp-lt female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'hisp-lt') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['H_FEMALE']        
        #native-am male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'native-am') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_MALE'] + row['NHNA_MALE']  
        #native-am female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'native-am') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHIA_FEMALE'] + row['NHNA_FEMALE']       
        #other male
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'other') & (pop['gender'] == 'male')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_MALE']        
        #other female
        i = pop.index[(pop['county_fips_code'].astype(int) == county) & (pop['aggrp'] == '65+') & (pop['race'] == 'other') & (pop['gender'] == 'female')].tolist()
        pop.loc[i[0],'population'] = int(pop.loc[i[0], 'population']) + row['NHTOM_FEMALE'] 
pop
        
        

,county_fips_code,year,aggrp,race,gender,population
0,1,2016,18-24,white,male,519
1,1,2016,18-24,white,female,436
2,1,2016,18-24,black,male,210
3,1,2016,18-24,black,female,155
4,1,2016,18-24,asia-pi,male,5
...,...,...,...,...,...,...
19075,321,2016,65+,hisp-lt,female,22
19076,321,2016,65+,native-am,male,13
19077,321,2016,65+,native-am,female,4
19078,321,2016,65+,other,male,10


In [96]:
pop.to_excel("population.xlsx")  